In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
seed = 2019

from sklearn.preprocessing import StandardScaler

In [2]:
train_data = pd.read_csv('data/train.csv')
test_data=pd.read_csv('data/test.csv')

In [3]:
train_data.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
train_data.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [5]:
train_data.shape

(891, 12)

In [6]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
def preprocess(df):
    df = df.drop(['Name','Ticket', 'PassengerId', 'Cabin'], axis=1)  
    df.Age=df.Age.fillna(value=df.Age.median())
    df.Fare = df.Fare.fillna(value=df.Fare.median())
    df.Embarked = df.Embarked.fillna(value=df.Embarked.value_counts().idxmax())
    df.Sex = df.Sex.map({'female':1, 'male':0})
#     df.Cabin = df.Cabin.apply(lambda x: 0 if pd.isnull(x) else 1)
#     sex_one_hot = pd.get_dummies(df.Sex, prefix='Sex')
#     df = df.drop('Sex', axis=1)
#     df = df.join(sex_one_hot)
    embarked_one_hot = pd.get_dummies(df.Embarked, prefix='Embarked')
    df = df.drop('Embarked', axis=1)
    df = df.join(embarked_one_hot)
    return df

In [8]:
train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [9]:
train_data.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,0,0,1
1,1,1,1,38.0,1,0,71.2833,1,0,0
2,1,3,1,26.0,0,0,7.9250,0,0,1
3,1,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,0,35.0,0,0,8.0500,0,0,1
5,0,3,0,28.0,0,0,8.4583,0,1,0
6,0,1,0,54.0,0,0,51.8625,0,0,1
7,0,3,0,2.0,3,1,21.0750,0,0,1
8,1,3,1,27.0,0,2,11.1333,0,0,1
9,1,2,1,14.0,1,0,30.0708,1,0,0


In [10]:
train_data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X = train_data.drop(['Survived'], axis=1).values.astype(float)
Y = train_data.Survived.values
X_train, X_val, y_train, y_val = train_test_split(X, Y, 
                                                    test_size = .3,
                                                    random_state = 5,
                                                   stratify = Y)  # stratify 保证训练和测试集的分布相同

In [12]:
X_train[0]

array([ 3.    ,  0.    , 25.    ,  0.    ,  0.    ,  7.8958,  0.    ,
        0.    ,  1.    ])

In [28]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion='entropy', max_depth=4)
rf.fit(X_train, y_train)
rf.score(X_val, y_val)

c:\devsw\python\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8283582089552238

In [37]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=3)
xgb.fit(X_train, y_train)
xgb.score(X_val, y_val)

0.8208955223880597

In [38]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
lg.fit(X_train, y_train)
lg.score(X_val, y_val)

c:\devsw\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7798507462686567

In [16]:
# scale=StandardScaler()
# X = scale.fit_transform(X)

In [17]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Activation

def create_model(optimizer='adam', init='uniform'):
    # create model
    model = Sequential()
#     model.add(Dense(8, input_dim=X.shape[1], kernel_initializer=init))
#     model.add(BatchNormalization())
#     model.add(Activation(activation='relu'))
#     model.add(Dense(16, kernel_initializer=init))
#     model.add(BatchNormalization())
#     model.add(Activation(activation='relu'))
    model.add(Dense(8, kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
#     model.add(Dense(4, kernel_initializer=init))
#     model.add(BatchNormalization())
#     model.add(Activation(activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import time

run_gridsearch = False

if run_gridsearch:
    
    start_time = time.time()
    print (time.strftime( "%H:%M:%S " + "GridSearch started ... " ) )
    optimizers = ['rmsprop', 'adam']
    inits = ['glorot_uniform', 'normal', 'uniform']
    epochs = [50, 100, 200, 400]
    batches = [5, 10, 20]
    
    model = KerasClassifier(build_fn=create_model, verbose=0)
    
    param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
    grid = GridSearchCV(estimator=model, param_grid=param_grid)
    grid_result = grid.fit(X, Y)
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    elapsed_time = time.time() - start_time  
    print ("Time elapsed: ",timedelta(seconds=elapsed_time))
        
    best_epochs = grid_result.best_params_['epochs']
    best_batch_size = grid_result.best_params_['batch_size']
    best_init = grid_result.best_params_['init']
    best_optimizer = grid_result.best_params_['optimizer']
    
else:
    # pre-selected paramters
    best_epochs = 50
    best_batch_size = 10
    best_init = 'glorot_uniform'
    best_optimizer = 'rmsprop'

In [19]:
model_pred = KerasClassifier(build_fn=create_model, optimizer=best_optimizer, init=best_init, epochs=best_epochs, batch_size=best_batch_size, verbose=1)
# model_pred.fit(X, Y)

In [23]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators=[
    ('rf',rf),
    ('xgb', xgb),
    ('lg', lg),
#     ('nn', model_pred)
# https://github.com/scikit-learn/scikit-learn/issues/11031
])
vc.fit(X_train, y_train)

c:\devsw\python\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\devsw\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=4,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators='warn',
                                                     n_jobs=None,
       

In [26]:
# Create X_test
X_test = test_data.values.astype(float)
# Scaling
# X_test = scale.fit_transform(X_test)
# print(X_test.shape)
# Predict 'Survived'
prediction = vc.predict(X_test)
test_df = pd.read_csv('data/test.csv',index_col='PassengerId')
submission = pd.DataFrame({
    'PassengerId': test_df.index,
    'Survived': prediction,
})

submission.sort_values('PassengerId', inplace=True)    
submission.to_csv('submission-simple-cleansing.csv', index=False)

In [31]:
X_test = test_data.values.astype(float)
X_test[0]


array([ 3.    ,  0.    , 34.5   ,  0.    ,  0.    ,  7.8292,  0.    ,
        1.    ,  0.    ])

In [32]:
X_train[0]

array([ 3.    ,  0.    , 25.    ,  0.    ,  0.    ,  7.8958,  0.    ,
        0.    ,  1.    ])

In [43]:
y_train.shape

(623,)